In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Dense, Conv2D
from tensorflow.keras.models import Sequential

class HybridSECBAM(Layer):
    def __init__(self, se_ratio=16, cbam_ratio=8, **kwargs):
        super().__init__(**kwargs)
        self.se_ratio = se_ratio
        self.cbam_ratio = cbam_ratio

    def get_config(self):
        config = super().get_config()
        config.update({
            'se_ratio': self.se_ratio,
            'cbam_ratio': self.cbam_ratio,
        })
        return config

    def build(self, input_shape):
        channels = input_shape[-1]
        
        # SE layers
        self.se_dense1 = Dense(channels // self.se_ratio, activation='swish')
        self.se_dense2 = Dense(channels, activation='sigmoid')

        # CBAM channel attention MLP (shared for avg and max)
        self.channel_mlp = Sequential([
            Dense(channels // self.cbam_ratio, activation='swish'),
            Dense(channels)
        ])
        
        # CBAM spatial attention
        self.spatial_conv = Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, inputs):
      dtype = inputs.dtype  # Match all tensors to this dtype

       # --- SE ---
      se = tf.reduce_mean(inputs, axis=[1, 2])
      se = self.se_dense1(se)
      se = self.se_dense2(se)
      se = tf.reshape(se, [-1, 1, 1, inputs.shape[-1]])
      se = tf.cast(se, dtype)

      # --- CBAM Channel Attention ---
      avg_pool = tf.reduce_mean(inputs, axis=[1, 2])
      max_pool = tf.reduce_max(inputs, axis=[1, 2])
      channel_att = tf.nn.sigmoid(self.channel_mlp(avg_pool) + self.channel_mlp(max_pool))
      channel_att = tf.reshape(channel_att, [-1, 1, 1, inputs.shape[-1]])
      channel_att = tf.cast(channel_att, dtype)

      # --- CBAM Spatial Attention ---
      avg_pool_spatial = tf.reduce_mean(inputs, axis=-1, keepdims=True)
      max_pool_spatial = tf.reduce_max(inputs, axis=-1, keepdims=True)
      concat = tf.concat([avg_pool_spatial, max_pool_spatial], axis=-1)
      spatial_att = self.spatial_conv(concat)
      spatial_att = tf.cast(spatial_att, dtype)

      # --- Combine ---
      out = inputs * se * channel_att * spatial_att
      return out



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, MultiHeadAttention, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import GroupNormalization
from tensorflow.keras import layers

class EnhancedMHABlock(Layer):
    def __init__(self, num_heads=8, key_dim=64, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim

    def build(self, input_shape):
        channels = input_shape[-1]
        if channels is None:
            raise ValueError("Input channels must be defined.")

        self.mha = layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.key_dim,
            kernel_initializer='glorot_uniform',
            bias_initializer='zeros'
        )

        self.ffn = Sequential([
            Conv2D(4 * channels, kernel_size=1, activation='swish'),
            Conv2D(channels, kernel_size=1)
        ])

        def get_valid_group_count(ch):
            for g in reversed(range(1, 33)):
                if ch % g == 0:
                    return g
            return 1

        group_count = get_valid_group_count(channels)

        self.norm1 = GroupNormalization(groups=group_count)
        self.norm2 = GroupNormalization(groups=group_count)

    def call(self, inputs):
       dtype = inputs.dtype  # Consistently use the same dtype throughout

       shape = tf.shape(inputs)
       B, H, W, C = shape[0], shape[1], shape[2], shape[3]

       x_flat = tf.reshape(inputs, [B, H * W, C])
       attn = self.mha(x_flat, x_flat)
       attn = tf.reshape(attn, [B, H, W, C])
       attn = tf.cast(attn, dtype)

       x = self.norm1(inputs + attn)
       ffn_out = self.ffn(x)
       return self.norm2(x + ffn_out)


    def get_config(self):
        config = super().get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
        })
        return config


In [ ]:
from keras.applications import EfficientNetB7
from tensorflow.keras import layers, models
from tensorflow.keras import mixed_precision
import tensorflow as tf

def build_enhanced_model(img_size=(300, 300), num_classes=1):
    base_model = EfficientNetB7(
        weights='imagenet',
        include_top=False,
        input_shape=(*img_size, 3)
    )

    # Feature extraction
    block3_output = base_model.get_layer('block3a_project_bn').output
    block5_output = base_model.get_layer('block5a_project_bn').output

    # Attention pipeline
    x3 = HybridSECBAM()(block3_output)
    x3 = EnhancedMHABlock()(x3)
    x3 = layers.MaxPooling2D(pool_size=2)(x3)

    x5 = HybridSECBAM()(block5_output)
    x5 = EnhancedMHABlock()(x5)

    x = layers.Concatenate()([x3, x5])

    max_pool = layers.GlobalMaxPooling2D()(x)
    avg_pool = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([max_pool, avg_pool])

    x = layers.Dense(512, activation='swish', dtype='float32')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='swish', dtype='float32')(x)
    output = layers.Dense(num_classes, activation='sigmoid', dtype='float32')(x)

    model = models.Model(inputs=base_model.input, outputs=output)
    return model


# Step 1: Make sure mixed precision is NOT enabled before model creation
mixed_precision.set_global_policy('float32')

# Step 2: Build the model
model = build_enhanced_model(img_size=(224, 224), num_classes=1)

# Step 3: Enable mixed precision globally (only after model is built)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Step 4: Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)